In [15]:
mapping = {
    "Freezing": 0,
    "Warm": 1,
    "Cold": 2,
    "Boiling Hot": 3,
    "Hot": 4,
    "Lava Hot": 5
}

In [32]:
import pandas as pd
df = pd.read_csv("../input/cat_train.csv")

# df.loc[:, "ord_2"] = df.ord_2.map(mapping)

In [35]:
df.ord_2.value_counts()

Freezing       142726
Warm           124239
Cold            97822
Boiling Hot     84790
Hot             67508
Lava Hot        64840
Name: ord_2, dtype: int64

In [18]:
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv("../input/cat_train.csv")

df.loc[:, "ord_2"] = df.ord_2.fillna("NONE")

lbl_enc = preprocessing.LabelEncoder()

df.loc[:, "ord_2"] = lbl_enc.fit_transform(df.ord_2.values)


In [19]:
df.ord_2.value_counts()

2    142726
6    124239
1     97822
0     84790
3     67508
4     64840
5     18075
Name: ord_2, dtype: int64

In [20]:
import numpy as np
from scipy import sparse

n_rows = 10000

n_cols = 100000

example = np.random.binomial(1, p=0.05, size=(n_rows, n_cols))

print(f"Size of dense array : {example.nbytes}")

sparse_example = sparse.csr_matrix(example)

print(f"Size of sparse array : {sparse_example.data.nbytes}")

full_size = (
    sparse_example.data.nbytes + 
    sparse_example.indptr.nbytes +
    sparse_example.indices.nbytes
)

print(f"Full size of sparse array : {full_size}")


Size of dense array : 4000000000
Size of sparse array : 200041352
Full size of sparse array : 400122708


In [3]:
import numpy as np
from scipy import sparse


example = np.array(
    [
        [0,0,0,0,1,0],
        [0,1,0,0,0,0],
        [1,0,0,0,0,0]
    ]
)

print(f"Size of dense array : {example.nbytes}")

sparse_example = sparse.csr_matrix(example)

print(f"Size of sparse array : {sparse_example.data.nbytes}")

full_size = (
    sparse_example.data.nbytes + 
    sparse_example.indptr.nbytes +
    sparse_example.indices.nbytes
)

print(f"Full size of sparse array : {full_size}")

Size of dense array : 72
Size of sparse array : 12
Full size of sparse array : 40


In [6]:
import numpy as np
from sklearn import preprocessing

example = np.random.randint(1000, size=100000)

ohe = preprocessing.OneHotEncoder(sparse=False)

ohe_example = ohe.fit_transform(example.reshape(-1, 1))
print(f"Size of dense array : {ohe_example.nbytes}")

ohe = preprocessing.OneHotEncoder(sparse=True)

ohe_example = ohe.fit_transform(example.reshape(-1, 1))
print(f"Size of sparse array : {ohe_example.data.nbytes}")

full_size = (
    ohe_example.data.nbytes +
    ohe_example.indptr.nbytes + 
    ohe_example.indices.nbytes
)

print(f"Full size of sparse array : {full_size}")


Size of dense array : 800000000
Size of sparse array : 800000
Full size of sparse array : 1600004


In [26]:
df.groupby(["ord_2"])["id"].count()

ord_2
0     84790
1     97822
2    142726
3     67508
4     64840
5     18075
6    124239
Name: id, dtype: int64

In [27]:
df.groupby(["ord_2"])["id"].transform("count")

0          67508
1         124239
2         142726
3          64840
4          97822
           ...  
599995    142726
599996     84790
599997    142726
599998    124239
599999     84790
Name: id, Length: 600000, dtype: int64

In [28]:
df.groupby(
    [
        "ord_1",
        "ord_2"
    ]
)["id"].count().reset_index(name="count")

,ord_1,ord_2,count
0,Contributor,0,15634
1,Contributor,1,17734
2,Contributor,2,26082
3,Contributor,3,12428
4,Contributor,4,11919
5,Contributor,5,3250
6,Contributor,6,22774
7,Expert,0,19477
8,Expert,1,22956
9,Expert,2,33249


In [34]:
df["new_feature"] = (
    df.ord_1.astype(str)
    + "_"
    + df.ord_2.astype(str)
    + "_"
    + df.ord_3.astype(str)
)

df.new_feature

0                 Contributor_Hot_c
1                Grandmaster_Warm_e
2                    nan_Freezing_n
3                 Novice_Lava Hot_a
4                Grandmaster_Cold_h
                    ...            
599995            Novice_Freezing_a
599996         Novice_Boiling Hot_n
599997       Contributor_Freezing_n
599998                Master_Warm_m
599999    Contributor_Boiling Hot_b
Name: new_feature, Length: 600000, dtype: object

In [37]:
df.ord_2.fillna("NONE").value_counts()

Freezing       142726
Warm           124239
Cold            97822
Boiling Hot     84790
Hot             67508
Lava Hot        64840
NONE            18075
Name: ord_2, dtype: int64

In [38]:
import pandas as pd
from sklearn import preprocessing

train = pd.read_csv("../input/cat_train.csv")

test = pd.read_csv("../input/cat_test.csv")

test.loc[:, "target"] = -1

data = pd.concat([train, test]).reset_index(drop=True)

features = [x for x in train.columns if x not in ["id", "target"]]

for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    
    temp_col = data[feat].fillna("NONE").astype(str).values
    
    data.loc[:, feat] = lbl_enc.fit_transform(temp_col)

train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)

In [39]:
df.ord_2.fillna("NONE").value_counts()

Freezing       142726
Warm           124239
Cold            97822
Boiling Hot     84790
Hot             67508
Lava Hot        64840
NONE            18075
Name: ord_2, dtype: int64

In [40]:
df.ord_4.fillna("NONE").value_counts()

N       39978
P       37890
Y       36657
A       36633
R       33045
U       32897
M       32504
X       32347
C       32112
H       31189
Q       30145
T       29723
O       25610
B       25212
E       21871
K       21676
I       19805
NONE    17930
D       17284
F       16721
W        8268
Z        5790
S        4595
G        3404
V        3107
J        1950
L        1657
Name: ord_4, dtype: int64

In [41]:
df.ord_4 = df.ord_4.fillna("NONE")

In [48]:
df.loc[
    df["ord_4"].value_counts()[df["ord_4"]].values < 2000,
    "ord_4"
] = "RARE"

df.ord_4.value_counts()

ValueError: cannot index with vector containing NA / NaN values

In [44]:
import pandas as pd

df = pd.read_csv("../input/cat_train_folds.csv")

df.kfold.value_counts()

4    120000
3    120000
2    120000
1    120000
0    120000
Name: kfold, dtype: int64

In [78]:
for i in range(0, 5):
    print(df[df.kfold==i].target.value_counts())

0    97536
1    22464
Name: target, dtype: int64
0    97536
1    22464
Name: target, dtype: int64
0    97535
1    22465
Name: target, dtype: int64
0    97535
1    22465
Name: target, dtype: int64
0    97535
1    22465
Name: target, dtype: int64


In [77]:
!python src/create_folds.py

Traceback (most recent call last):
  File "src/create_folds.py", line 7, in <module>
    df = pd.read_csv("../input/cat_test.csv")
  File "D:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "D:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 457, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "D:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "D:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 1135, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "D:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 1917, in __init__
    self._reader = parsers.TextReader(src, **kwds)
  File "pandas/_libs/parsers.pyx", line 382, in pandas._libs.parsers.TextReader.__cinit__
  File "pandas/_libs/parsers.pyx", line 689, in pandas._libs.parsers.TextReader._setup_parser_source
FileNotFoundError: [E